# Matemáticas para el tratamiento de datos
Autor: Adrián Robles Arques

Objetivo: Generar un algoritmo de recomendación de películas a usuarios basadas en similitudes entre las que ya han visto y le han gustado.


# 1. Leer datos desde Excel

Pandas dispone de funciones para leer datos desde formatos de tabla de datos, como xlsx, que es el caso. Dicha herramienta cuenta con la posibilidad de leer diversas hojas, o las hojas indicadas, dentro de la tabla de datos objetivo.

In [1]:
#Primero importamos las librerías necesarias
import pandas as pd
import numpy as np
import scipy as sp

#Importamos librerias para cargar datos en colab
from google.colab import files
import io

#librerías para escribir datos en documentos
from csv import writer

In [4]:
from os import name
#Cargamos los datos desde local
uploaded = {}
uploaded = files.upload()

Saving plantilla_peliculas.xlsx to plantilla_peliculas (1).xlsx


In [6]:
#Leemos el nombre del archivo, por si se carga varias veces.
names = []
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  names.append(fn)

User uploaded file "plantilla_peliculas (1).xlsx" with length 203436 bytes


In [10]:
#Creamos los DataFrames con las 3 primeras hojas
df_data = pd.read_excel(io.BytesIO(uploaded[names[0]]), sheet_name= [0,1,2])

In [17]:
#Separamos los diferentes Dataframes
df_movies = df_data[0]
df_ratings = df_data[1]
df_users = df_data[2][['userId', 'Género','Edad']]

In [18]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [19]:
df_ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [20]:
df_users.head()

,userId,Género,Edad
0,1,H,29
1,3,M,24
2,4,M,29
3,5,H,32
4,6,M,18


Ahora vamos a analizar los géneros de cada películas, a dividirlos y a codificarlos en forma one hot, para después poder hacer el análisis vectorial.

In [21]:
#Separamos los géneros en listas
df_movies['genres'] = df_movies['genres'].str.split('|')

In [22]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [36]:
#Hacemos la codificación one_hot manualmente dado que trabajamos con una lista de géneros

#Primero vamos a extraer la lista del total de géneros diferentes
genres_list = []
for index, row in df_movies.iterrows():
  for genre in row['genres']:
    if genre not in genres_list:
      genres_list.append(genre)

#Ahora vamos a iterar por género, comprobando si aparece en la lista de géneros de la película
for genre in genres_list:
  aux_list = []
  for index, row in df_movies.iterrows():
    aux_list.append(int(genre in row['genres']))
  df_movies[genre] = aux_list

df_movies.head()

,movieId,title,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),[Comedy],0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [161]:
df_movies.set_index('movieId', inplace=True)
df_movies.head()

,title,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,vector_gen
movieId,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,"[0.4472135954999579, 0.4472135954999579, 0.447..."
2,Jumanji (1995),"[Adventure, Children, Fantasy]",1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,"[0.5773502691896258, 0.0, 0.5773502691896258, ..."
3,Grumpier Old Men (1995),"[Comedy, Romance]",0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,"[0.0, 0.0, 0.0, 0.7071067811865475, 0.0, 0.707..."
4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,"[0.0, 0.0, 0.0, 0.5773502691896258, 0.0, 0.577..."
5,Father of the Bride Part II (1995),[Comedy],0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


Vamos a escribir la representación vectorial en el excel

In [162]:
with pd.ExcelWriter(names[0],mode = 'a', if_sheet_exists = 'overlay') as writer:
  df_movies[genres_list].to_excel(writer, sheet_name = 'Representación vectorial')

Una vez realizado el tratamiento de los datos y la codificación como la queríamos, vamos a evaluar la similitud entre todas las películas, atendiendo al método del coseno. Para ello vamos a generar un vector unitario que codifique los géneros a los que pertence cada película y comprobaremos su cercanía al resto de películas para cada una de la lista.

In [45]:
#Vamos a generar los vectores unitarios para cada película

df_movies['vector_gen'] = df_movies[genres_list].values.tolist()
df_movies['vector_gen'] = df_movies['vector_gen'].apply(lambda x: np.array(x) / np.linalg.norm(x))

df_movies.head()

,movieId,title,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,vector_gen
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,"[0.4472135954999579, 0.4472135954999579, 0.447..."
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,"[0.5773502691896258, 0.0, 0.5773502691896258, ..."
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,"[0.0, 0.0, 0.0, 0.7071067811865475, 0.0, 0.707..."
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,"[0.0, 0.0, 0.0, 0.5773502691896258, 0.0, 0.577..."
4,5,Father of the Bride Part II (1995),[Comedy],0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


Vamos a escribir los vectores normalizados en el excel

In [169]:
#Creamos un nuevo dataframe con los vectores normalizados
df_norm_vectors = pd.DataFrame(df_movies['vector_gen'].tolist(), index=df_movies.index)
df_norm_vectors['norma'] = [np.linalg.norm(x) for x in
                            df_movies[genres_list].values.tolist()]

#Escribimos el resultado en la hoja correspondiente
with pd.ExcelWriter(names[0],mode = 'a', if_sheet_exists = 'overlay') as writer:
  df_norm_vectors.to_excel(writer, sheet_name = 'Normalización de vectores')

A continuación vamos a calcular la similitud de las peliculas entre si, usando para ello la regla del coseno. Dado que los vectores ya están normalizados, únicamente debemos calcular el producto vectorial entre todos los vectores asociados a cada película.

In [62]:
#Creamos un nuevo dataframe con los productos vectoriales de todos los vectores
product_dict = {}
for index, row in df_movies.iterrows():
  product_dict[row['movieId']] = np.dot(row['vector_gen'], np.array(df_movies['vector_gen'].tolist()).T)

df_products = pd.DataFrame(product_dict, index=df_movies['movieId'].tolist())
df_products.head()

,1,2,3,4,5,6,7,8,9,10,...,102,103,104,105,106,107,108,110,111,112
1,1.000000,0.774597,0.316228,0.258199,0.447214,0.0,0.316228,0.632456,0.0,0.258199,...,0.447214,0.0,0.447214,0.000000,0.316228,0.670820,0.0,0.000000,0.000000,0.447214
2,0.774597,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.816497,0.0,0.333333,...,0.000000,0.0,0.000000,0.000000,0.000000,0.577350,0.0,0.000000,0.000000,0.288675
3,0.316228,0.000000,1.000000,0.816497,0.707107,0.0,1.000000,0.000000,0.0,0.000000,...,0.707107,0.0,0.707107,0.500000,0.500000,0.353553,0.0,0.000000,0.000000,0.353553
4,0.258199,0.000000,0.816497,1.000000,0.577350,0.0,0.816497,0.000000,0.0,0.000000,...,0.577350,0.0,0.577350,0.816497,0.816497,0.288675,0.0,0.333333,0.333333,0.288675
5,0.447214,0.000000,0.707107,0.577350,1.000000,0.0,0.707107,0.000000,0.0,0.000000,...,1.000000,0.0,1.000000,0.000000,0.707107,0.500000,0.0,0.000000,0.000000,0.500000


Vamos a escribir los resultados en el excel, para ello emplearemos las funciones de pandas ExcelWriter y to_excel.

In [171]:
with pd.ExcelWriter(names[0],mode = 'a', if_sheet_exists = 'overlay') as writer:
  df_products.to_excel(writer, sheet_name = 'Matriz de Similitudes (coseno)')

Teniendo ya la matriz de similitudes, necesitamos evaluar qué peliculas ha visto cada usuario y la puntiación atribuida a estas películas vistas, para poder predecir qué pelicula de las que no ha visto tiene más probabilidad de gustarle.

In [63]:
df_ratings.loc[df_ratings['userId'] == 1]

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
5,1,70,3.0
6,1,101,5.0
7,1,110,4.0


In [107]:
#Vamos a definir una función que nos determine una predicción del score
#Esta función se basará en las peliculas vistas por cada usuario y la puntiación dada
#Así como la similitud de esta película con las demás

def pred_score(user_id, movie_id):
  df_user = df_ratings.loc[df_ratings['userId'] == user_id]
  if movie_id not in df_user['movieId'].tolist():
    p_score = 0.0
    for index, row in df_user.iterrows():
      p_score += row['rating'] * df_products.loc[movie_id][row['movieId']]
  else:
    p_score = 0.0 #Queremos que ignore películas ya vistas

  return p_score


In [108]:
#Prueba
pred_score(610, 1)

0.0

In [133]:
#Con la función elaborada, vamos a crear una matriz de predicciones por pelicula y usuario
userId_list = df_ratings['userId'].unique()
movieId_list = df_ratings['movieId'].unique()

#Generamos la matrix de datos
pred_list = []
for user in userId_list:
  pred_user = []
  for movie in movieId_list:
    pred = pred_score(user, movie)
    if pred == np.NaN:
      print('NaN')
    pred_user.append(pred)
  pred_list.append(pred_user)

#Convertimos datos a dataframe
df_pred = pd.DataFrame(pred_list, index=userId_list, columns= movieId_list)

In [134]:
df_pred.head()

,1,3,6,47,50,70,101,110,31,21,...,83,63,42,40,30,80,96,103,77,108
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.309401,15.325832,...,5.400760,11.535340,9.309401,2.309401,7.074995,2.897904,7.726331,9.615175,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.288675,0.000000,0.000000,...,0.353553,0.000000,0.288675,0.500000,0.353553,0.353553,0.353553,0.000000,0.0,0.0
4,4.363298,9.348469,4.483163,0.000000,5.966326,7.894823,7.476391,4.632993,8.024580,0.000000,...,8.123724,8.123724,5.632993,8.024580,6.898979,5.674235,12.573214,5.632993,0.0,0.0
5,0.000000,9.980387,8.299660,4.898979,0.000000,8.017265,11.675241,0.000000,10.853007,0.000000,...,11.215476,11.705132,11.898979,10.853007,12.940221,10.939146,14.113380,4.000000,0.0,0.0
6,37.558613,0.000000,0.000000,0.000000,0.000000,51.574509,49.413141,0.000000,0.000000,0.000000,...,63.609424,38.320092,59.454790,65.033600,56.338790,58.976431,73.952701,29.196444,0.0,0.0


In [135]:
df_pred.tail()

,1,3,6,47,50,70,101,110,31,21,...,83,63,42,40,30,80,96,103,77,108
606,0.0,28.775272,15.129682,0.000000,0.000000,0.000000,26.469639,0.0,33.105497,23.192554,...,38.037480,19.162900,26.697037,33.105497,28.425108,0.000000,37.556035,15.261314,0.0,0.0
607,0.0,5.921508,5.787694,0.000000,2.210343,5.358529,8.650479,0.0,13.999271,4.820489,...,12.623724,5.903869,13.870177,13.999271,12.606086,12.663891,13.095742,0.000000,0.0,0.0
608,0.0,0.000000,19.610410,0.000000,0.000000,0.000000,21.792819,0.0,0.000000,0.000000,...,13.815907,0.000000,18.350900,14.336126,17.127903,14.713012,24.966066,16.023423,0.0,0.0
609,0.0,0.948683,3.666667,1.632993,1.333333,3.846247,2.496341,0.0,1.732051,2.107930,...,1.224745,0.948683,3.333333,1.732051,1.224745,2.173428,2.173428,1.333333,0.0,0.0
610,0.0,4.409566,0.000000,0.000000,0.000000,0.000000,12.878873,0.0,8.666808,21.788155,...,6.128359,13.789256,19.305003,8.666808,15.508048,7.709498,10.537925,16.903850,0.0,0.0


Ahora vamos a añadir el dato del título de cada película, para ubicarla junto al userId y el movieId.

In [138]:
#Localizamos los títulos para cada Id
movie_titles = [df_movies.loc[df_movies['movieId'] == movie]['title'].values[0]
                for movie in movieId_list]

dfpred = df_pred.T #Creamos un Df nuevo, así no perdemos el que únicamente tiene valores numéricos
dfpred['title'] = movie_titles
dfpred.head()

,1,3,4,5,6,7,8,9,11,12,...,602,603,604,605,606,607,608,609,610,title
1,0.0,0.0,4.363298,0.000000,37.558613,0.000000,8.210198,0.0,2.581989,1.264911,...,12.112357,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,Toy Story (1995)
3,0.0,0.0,9.348469,9.980387,0.000000,3.872515,7.898979,0.0,0.000000,4.000000,...,16.987428,28.361986,14.812644,3.264911,28.775272,5.921508,0.00000,0.948683,4.409566,Grumpier Old Men (1995)
6,0.0,0.0,4.483163,8.299660,0.000000,3.000000,10.966326,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,15.129682,5.787694,19.61041,3.666667,0.000000,Heat (1995)
47,0.0,0.0,0.000000,4.898979,0.000000,3.674235,0.000000,0.0,4.490731,0.000000,...,0.000000,11.844628,14.144815,0.000000,0.000000,0.000000,0.00000,1.632993,0.000000,Seven (a.k.a. Se7en) (1995)
50,0.0,0.0,5.966326,0.000000,0.000000,0.000000,0.000000,0.0,6.966326,0.000000,...,0.000000,23.179139,17.145157,0.000000,0.000000,2.210343,0.00000,1.333333,0.000000,"Usual Suspects, The (1995)"


Vamos a crear una función de recomendaciones que devuelva un DataFrame con el top X de películas a recomendar a cada usuario, con datos del título e Id de la película.

In [174]:
#Generamos la función de recomendación
def recommend(user_id, top = 10):
  recomendation = dfpred[[user_id, 'title']].sort_values(by = user_id,
                                                         ascending=False)[0:top]
  recomendation.columns = ['Score', 'Titulo']
  recomendation['MovieId'] = recomendation.index
  recomendation.index = ['Top ' + str(n) for n in
                         np.arange(1, len(recomendation) + 1)]

  return recomendation

# Top 10 recomendaciones para el UserId 4
recommend(4)

,Score,Titulo,MovieId
Top 1,12.573214,Big Bully (1996),75
Top 2,12.573214,Antonia's Line (Antonia) (1995),82
Top 3,12.573214,Kicking and Screaming (1995),72
Top 4,12.573214,In the Bleak Midwinter (1995),96
Top 5,12.265986,Beautiful Girls (1996),94
Top 6,12.265986,"American President, The (1995)",11
Top 7,12.265986,Waiting to Exhale (1995),4
Top 8,11.424642,Money Train (1995),20
Top 9,9.940221,City Hall (1996),100
Top 10,9.940221,"Juror, The (1996)",79


In [175]:
#Escribimos la recomendación en el excel
with pd.ExcelWriter(names[0],mode = 'a', if_sheet_exists = 'overlay') as writer:
  recommend(4).to_excel(writer, sheet_name = 'Recomendaciones', startrow= 1)

Por último, descargamos el excel escrito

In [176]:
files.download(names[0])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>